### This notebook is created to create appropriate data for the subject wise engagement analysis. It creates a seperate dataframe for raw metrics, unnormalized metrics and log-normalized metrics. This data will be used to create vizualizations in the web-app

#### NOTE: At the end of notebook, we also have done similar analysis for overall discourse engagement.

In [1]:
import pandas as pd
import numpy as np

# Creating metrics for each subject

In [2]:
action_to_description = {
"1": "likes_given",
"2": "likes_received",
"3": "bookmarked_post",
"4": "created_new_topic",
"5": "replied",
"6": "received_response",
"7": "user_was_mentioned",
"9": "user's_post_quoted",
"11": "user_edited_post",
"12": "user_sent_private_message",
"13": "recieved_a_private_message",
"15": "solved_a_topic",
"16": "user_was_assigned",
"17": "linked"
}
def create_raw_metrics_dataframe(df):
    # Change the values in action_name column based on values of action_type and map it via the action_to_description dictionary. This is done to make the column_names more intuitive to understand.
    df['action_type'] = df['action_type'].astype(str)
    df['action_name'] = df['action_type'].map(action_to_description)
    df = pd.crosstab(df["acting_username"], df["action_name"]) # Pivot table

    columns_to_be_dropped = ['linked','received_response', "user's_post_quoted",
        'user_edited_post', 'user_was_mentioned'] # dropping columns which are not required for analysis

    df.drop(columns_to_be_dropped, axis=1, inplace=True, errors='ignore')
    subject_dataframe = df.copy()
    subject_dataframe['acting_username'] = subject_dataframe.index # Changing the index to a column
    subject_dataframe = subject_dataframe[["acting_username"]+[col for col in subject_dataframe.columns if col != 'acting_username']]  # Reordering the columns
    subject_dataframe.index = range(0, len(subject_dataframe))
    subject_dataframe.columns.name = None
    return subject_dataframe # Returns raw metrics dataframe

In [3]:
# Assign the weights to the relevant columns. This can be changed as per the requirement.
weights_dict = { 'likes_given': 0.3, # 0.3
                "likes_received": 0.8, # changed from 0.7
                "created_new_topic": 0.5, # changed from 1.0
                "replied": 0.7,
                'solved_a_topic': 1
}

def create_raw_scores_dataframe(df):
    columns_to_be_ignored = ["initial_score",'username','overall_topics_count_of_this_subject', 'normalised_score', 'z_score', "acting_username"] # this was directly taken from previous notebook. Some column names might seem irrelevant. Please ignore them.

    df["initial_score"] = sum(df[column]*weights_dict[column] for column in df.columns if column not in columns_to_be_ignored) # Initial score = sum(column_value*weight)

    df["z_score"] = round((df["initial_score"] - df["initial_score"].mean()) / df["initial_score"].std(),2) # z_score rounded to 2 decimal places
    return df.sort_values(by="z_score",ascending=False)

def create_log_normalized_scores_dataframe(df):
    # Apply log normalization to the numerical features
    numerical_features = df.select_dtypes(include=['number']).columns
    log_normalized_dataframe = df.copy()
    for feature in numerical_features:
        log_normalized_dataframe[feature] = round(np.log1p(log_normalized_dataframe[feature]),3)
    return log_normalized_dataframe.sort_values(by="z_score",ascending=False)


# Getting category_ids of courses

In [ ]:
# Get all the category_IDs
df = pd.read_csv("all_category_ids.csv")
ids_for_demo = [22,25,27,28,29]
df= df[df["category_id"].isin(ids_for_demo)]

import pandas as pd

for item in df.itertuples(name=None):
    index = item[0]
    category_id = item[1]
    category_name = item[2]

    user_actions_dataframe = pd.read_excel(f"course_excel_data/{category_name}.xlsx", sheet_name="user_actions_data")

    raw_metrics_dataframe = create_raw_metrics_dataframe(user_actions_dataframe)
    raw_scores_dataframe = create_raw_scores_dataframe(raw_metrics_dataframe)
    log_normalized_scores_dataframe = create_log_normalized_scores_dataframe(raw_metrics_dataframe)

    # Use ExcelWriter in 'openpyxl' mode to append multiple sheets
    file_path = f"course_excel_data_t1_2024/{category_name}.xlsx"
    with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
        raw_metrics_dataframe.to_excel(writer, sheet_name="raw_metrics", index=False)
        raw_scores_dataframe.to_excel(writer, sheet_name="unnormalized_scores", index=False)
        log_normalized_scores_dataframe.to_excel(writer, sheet_name="log_normalized_scores", index=False)

print("Excel files created with multiple sheets successfully!")


Excel files created with multiple sheets successfully!


# Analysis of overall engagement

In [11]:
# Assign the weights to the relevant columns. This can be changed as per the requirement.
weights_dict = { 'likes_given': 0.3, # 0.3
                "likes_received": 0.8, # changed from 0.7
                "topics_viewed": 0.4, # changed from 1.0
                "posts_read": 0.7,
                "days_visited": 0.7,
                'solutions': 1,
                "cheers": 0.3,
}

def create_raw_scores_dataframe_for_all_users(df):

    df["initial_score"] = sum(df[column]*weights_dict[column] for column in df.columns if column not in ["user_id"]) # Initial score = sum(column_value*weight)

    df["z_score"] = round((df["initial_score"] - df["initial_score"].mean()) / df["initial_score"].std(),2) # z_score rounded to 2 decimal places
    return df.sort_values(by="z_score",ascending=False)

def create_log_normalized_scores_dataframe(df):
    # Apply log normalization to the numerical features
    numerical_features = df.select_dtypes(include=['number']).columns
    log_normalized_dataframe = df.copy()
    for feature in numerical_features:
        log_normalized_dataframe[feature] = round(np.log1p(log_normalized_dataframe[feature]),3)
    return log_normalized_dataframe.sort_values(by="z_score",ascending=False)

In [14]:
user_actions_dataframe = pd.read_excel(f"course_excel_data_t1_2024/data_all_users.xlsx", sheet_name="user_actions_data")

raw_scores_dataframe = create_raw_scores_dataframe_for_all_users(user_actions_dataframe)
# log_normalized_scores_dataframe = create_log_normalized_scores_dataframe(raw_metrics_dataframe)

# Use ExcelWriter in 'openpyxl' mode to append multiple sheets
file_path = f"course_excel_data_t1_2024/data_all_users.xlsx"
with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    # raw_metrics_dataframe.to_excel(writer, sheet_name="raw_metrics", index=False)
    raw_scores_dataframe.to_excel(writer, sheet_name="unnormalized_scores", index=False)
    # log_normalized_scores_dataframe.to_excel(writer, sheet_name="log_normalized_scores", index=False)